In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder 
import os
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet
import string
nltk.download('averaged_perceptron_tagger')
from datetime import datetime

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/xenang/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
#combine Excel
hosp_lists = os.listdir("OCR")
hospitalname = []

for i in hosp_lists:
    sep = i.split()
    sep2 = sep[3].split(".")
    hospitalname.append(sep2[0])
hospitalname


combined_df = pd.DataFrame()

for hospitals in hosp_lists:
    excel_file_path = "OCR/" + hospitals

    all_sheets = pd.read_excel(excel_file_path, sheet_name=None)
    sep = hospitals.split()
    sep2 = sep[3].split(".")
    hospitalname = sep2[0]


    for sheet_name, sheet_df in all_sheets.items():
        sheet_df['Codes'] = sheet_name
        sheet_df['Hospitals'] = hospitalname
        try:
            combined_df = pd.concat([combined_df, sheet_df], ignore_index=True)
        except ValueError as e:
            print(f"Error processing sheet '{sheet_name}': {e}")

ocr_excel = combined_df

In [3]:
#normalise
codes = ocr_excel['Codes']
sn = ocr_excel['S/N']
gender = ocr_excel['Gender']
name = ocr_excel['NAME']

ocr_excel.drop(ocr_excel[((codes == '700') & (sn == 381)) | ((gender.isna()) & (name.isna()))].index, inplace=True)
gender = ocr_excel['Gender']
gender.loc[name.isin(['F', 'M'])] = gender[name.isin(['F', 'M'])].fillna(name)
name = ocr_excel['NAME']
name.replace('F', np.NaN, inplace=True)
name.replace('M', np.NaN, inplace=True)
ocr_excel['Type'].replace('OUPATIENT', 'OUTPATIENT', inplace=True)

/var/folders/sp/84j13ybj0zd_p4y410zpx7jc0000gn/T/ipykernel_11863/3336145435.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gender.loc[name.isin(['F', 'M'])] = gender[name.isin(['F', 'M'])].fillna(name)


In [4]:
#combine sheets
for index, row in ocr_excel.iterrows():
    if pd.isna(row['Gender']):
        matching_rows = ocr_excel.loc[codes == row['Codes']]
        for _, matching_row in matching_rows.iterrows():
            if not pd.isna(matching_row['Gender']):
                ocr_excel.at[index, 'DOB'] = matching_row['DOB']
                break

ocr_excel.loc[gender.isnull(), 'Gender'] = \
    ocr_excel.loc[gender.isnull(), 'Codes'].map(
        ocr_excel.dropna(subset=['Gender']).set_index('Codes')['Gender'].to_dict()
    )

#dob
def extract_year_from_date(date_string):
    if isinstance(date_string, int):
        return date_string
    date_obj = pd.to_datetime(date_string, errors='coerce')
    if not pd.isnull(date_obj):
        return date_obj.year
    else:
        return None

ocr_excel['DOB'] = ocr_excel['DOB'].apply(extract_year_from_date)
ocr_excel['DOB'] = ocr_excel['DOB'].fillna(ocr_excel.groupby('Codes')['DOB'].transform('last'))


current_year = datetime.now().year
ocr_excel['Age'] = current_year - ocr_excel['DOB']
dob_index = ocr_excel.columns.get_loc('DOB')
ocr_excel.insert(dob_index + 1, 'Age', ocr_excel.pop('Age'))
ocr_excel['Qty'].fillna(ocr_excel['Qty'], inplace = True)
ocr_excel['Subtotal GST'].fillna(ocr_excel['Subtotal']*1.07, inplace = True)
ocr_excel['Heading'].fillna(ocr_excel['Heading'], inplace = True)

combined_df = ocr_excel

#Normalise Date
combined_df['Date '] = pd.to_datetime(combined_df['Date '], errors='coerce')
combined_df = combined_df.dropna(subset=['Date '])
combined_df['Date '] = pd.to_datetime(combined_df['Date '])

combined_df['Year'] = combined_df['Date '].dt.year
combined_df['Month'] = combined_df['Date '].dt.month
combined_df['Day'] = combined_df['Date '].dt.day
combined_df = combined_df[(combined_df['Year'] >= 2016) & (combined_df['Year'] <= 2020)]

/var/folders/sp/84j13ybj0zd_p4y410zpx7jc0000gn/T/ipykernel_11863/1943261673.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_df['Date '] = pd.to_datetime(combined_df['Date '])
/var/folders/sp/84j13ybj0zd_p4y410zpx7jc0000gn/T/ipykernel_11863/1943261673.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_df['Year'] = combined_df['Date '].dt.year
/var/folders/sp/84j13ybj0zd_p4y410zpx7jc0000gn/T/ipykernel_11863/1943261673.py:45: SettingWithCopyWarning: 
A value is trying to be set on a co

In [5]:
#creating new columns to fit in model
def concat_columns(row):
    return ' '.join(str(item) for item in row)

combined_df['NLP'] = combined_df[['Provider', 'Heading', 'Code', 'Description', 'LOG Dx']].apply(concat_columns, axis=1)

descp = combined_df['NLP'].tolist()
unique_desc = list(set(descp))

cat = []

for i in unique_desc:
    s = i.split()
    if s[0] not in cat:
        cat.append(s[0])

In [6]:
# finding words end with "OMA"
get_ma_list = []
for i in unique_desc:
    s = i.split()
    for get_str in s:
       if get_str.endswith('oma') and get_str not in get_ma_list:
          get_ma_list.append(get_str)
          
get_ma_list = list(set([x.lower() for x in get_ma_list]))
get_ma_list = get_ma_list

In [7]:
# finding cancer names by MOH
get_excel_file = 'Input/cancer_list.xlsx'
cancer_list = pd.read_excel(get_excel_file)
cancer_word_list = []

for cancer_type in cancer_list['Cancer type'].dropna():
    if cancer_type not in cancer_word_list:
        cancer_word_list.append(cancer_type)

In [8]:
#Customisation of force adding words and remove common words
keywords =['medullablastoma','cancer','endocrine','hormonal','immunotherapy',"immunotherapy", "metastasis", "benign",
         "viopsy",'leukemia' ,'leukaemia' , "filgrastim", "cytology", "Ondansetron","lymphangiosarcoma"]

keyword = keywords + cancer_word_list + get_ma_list

lowercase = [str(item).lower() if isinstance(item, str) else item for item in keyword]
lowercase_list = lowercase
common_words = {'a','b', 'and', 'or', 'the','for', 'with', 'not', 'by', 'ii','iii','system','is', 'of', 
                'in', 'on', 'at', 'to','1','2','3','4','5','6','7','8','9','12','surgery','1st','2a','3b','complex',
                'c','t3','t4','treatment','type','during','only','core','exam','screening','others','others ','&stoma','stoma','glaucoma','myoma',
                'oncocare','chemo','chemotherapy','specimen','tumour','radiosurgery','oncocare','oncooare','invasive',
                'radiotherapy','transplantation','oncology','forceps'}

lowercase_list2 = [word for word in lowercase_list if word not in common_words]

def remove_duplicates(lst):
    seen = set()
    result = []
    for item in lst:
        if item not in seen:
            seen.add(item)
            result.append(item)
    return result

unique_list = remove_duplicates(lowercase_list2)

In [9]:
#Text mining / model formation
def remove_punctuation(text):
    return ''.join([char for char in text if char not in string.punctuation])

porter = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def process_text(text):
    tokens = word_tokenize(text)
    tokens = [remove_punctuation(token.lower()) for token in tokens if token.isalnum()]
    stemmed_tokens = [porter.stem(token) for token in tokens]
    lemmatized_tokens = [lemmatizer.lemmatize(token, get_wordnet_pos(token)) for token in tokens]
    return tokens, stemmed_tokens, lemmatized_tokens

def get_wordnet_pos(token):
    tag = nltk.pos_tag([token])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

#maiking list into string so can do manipulation later
def concatenate_list(my_list):
    concatenated_string = ""
    for element in my_list:
        concatenated_string += str(element) + " "
    return concatenated_string

#this code checks the words that contain in the cancer list
def check_keywords(tokens):
    cancerlist = []
    for token in tokens:
        if token in unique_list:
            cancerlist.append(token)
    return concatenate_list(cancerlist)


def is_empty(lst):
    if len(lst) == 0:
        return 0
    else:
        return 1
    
def combinecol (cola, colb):
    if cola == "cancer" and (colb != " " or colb != "" or colb.notna()):
        return colb
    elif cola == " " or cola == "":
        return colb
    else:
        return cola
    
def checkuniq(lst):
    splits = lst.split(" ")
    uniques = ['cancer','specimen','chemo','tumor']
    if len(splits) >= 1:
        for i in splits:
            if i in get_ma_list:
                return i
            elif i in uniques:
                split2 = [x for x in splits if x not in uniques]
                if len(split2) >= 1:
                    #print("one")
                    return splits[0]
                else:
                    #print("two")
                    return split2[1]
            else:
                #print("three")
                return splits[0]
            
    elif len(splits) == 1:
        #print("four")
        return "cancer"

#Text mining on medications
med = []

for cancer_type in cancer_list['Active ingredient'].dropna():
    splits = cancer_type.split()
    if splits[0] not in med:
        med.append(cancer_type)
medi = [item.lower() for item in med]

def check_keywords2(tokens):
    plist = ["and","511001", "4001000036","4001000038","4001000037",'the','novena','parkway','chemotherapy',
             'oncocare','chemo','chemotherapy','specimen','tumour','radiosurgery','oncocare','oncooare','invasive',
                'radiotherapy','transplantation','oncology','forceps','clinic']
    counter = 0 
    for token in tokens:
        if token == "cancer":
            counters = counter - 1
            if tokens[counters] not in plist:
                return tokens[counters]
        counter += 1

def check_keywords3(tokens):
    medilist = []
    for token in tokens:
        if token.lower() in medi:
            medilist.append(token)
    return concatenate_list(medilist)

def normalise(norm):
    if norm == "leukemia":
        return "leukaemia"
    elif norm == "esophageal":
        return "oesophageal"
    elif norm == "cholanglocarcinoma":
        return "cholangiocarcinoma"
    elif norm == "medullablastoma":
        return "medulloblastoma"
    elif norm == "tumor":
        return "tumour"
    elif norm == "ductal":
        return "breast"
    elif norm == "gluacoma":
        return "glaucoma"
    else:
        return norm

In [10]:
combined_df['Tokens'], combined_df['Stemmed'], combined_df['Lemmatized'] = zip(*combined_df['NLP'].apply(process_text))

In [11]:
"""
def check_keywords(tokens):
    cancerlist = []
    repeatlists = ['cancer','chemo','chemotherapy','biopsy']
    counter = 0
    result_list = []

    for token in tokens:
        if token in unique_list:
            cancerlist.append(token)

    for i in cancer_list:
        for r in repeatlists:
            if r in i:
                counter += 1
        if counter >=1:
            for item in cancerlist:
                if item not in repeatlists:
                    result_list.append(item)

            return result_list
        else:
            return cancer_list
"""

"\ndef check_keywords(tokens):\n    cancerlist = []\n    repeatlists = ['cancer','chemo','chemotherapy','biopsy']\n    counter = 0\n    result_list = []\n\n    for token in tokens:\n        if token in unique_list:\n            cancerlist.append(token)\n\n    for i in cancer_list:\n        for r in repeatlists:\n            if r in i:\n                counter += 1\n        if counter >=1:\n            for item in cancerlist:\n                if item not in repeatlists:\n                    result_list.append(item)\n\n            return result_list\n        else:\n            return cancer_list\n"

In [12]:
#results text mining
df_cleaned = combined_df

df_cleaned['Keywords_Present'] = df_cleaned['Tokens'].apply(check_keywords)
df_cleaned['Keywords_Present2'] = df_cleaned['Stemmed'].apply(check_keywords)
df_cleaned['Keywords_Present3'] = df_cleaned['Lemmatized'].apply(check_keywords)

df_cleaned['is_non_empty'] = df_cleaned['Keywords_Present'].apply(is_empty)
df_cleaned['is_non_empty2'] = df_cleaned['Keywords_Present2'].apply(is_empty)
df_cleaned['is_non_empty3'] = df_cleaned['Keywords_Present3'].apply(is_empty)

df_cleaned['Cancer'] = df_cleaned['is_non_empty'] + df_cleaned['is_non_empty2'] + df_cleaned['is_non_empty3']
df_cleaned['testing'] = df_cleaned['Keywords_Present'] +df_cleaned['Keywords_Present2'] +df_cleaned['Keywords_Present3']

df_cleaned['cancer2'] = df_cleaned['Tokens'].apply(check_keywords2)

df_cleaned['MainCancer'] = df_cleaned['testing'].apply(checkuniq)
df_cleaned['medicine2'] = df_cleaned['Lemmatized'].apply(check_keywords3)
df_cleaned['Combined'] = df_cleaned.apply(lambda row: combinecol(row['MainCancer'], row['cancer2']), axis=1)
df_cleaned['Patient'] = df_cleaned['Hospitals'].astype(str) + " - "+ df_cleaned['Codes'].astype(str)
df_cleaned['Patient'] = df_cleaned['Patient'].str.strip()
df_cleaned['CancerPatient'] = df_cleaned.groupby('Patient')['Combined'].transform(lambda x: 1 if x.notna().any() else 0)

In [13]:
df_cleaned['Combined'] = df_cleaned.apply(lambda row: combinecol(row['MainCancer'], row['cancer2']), axis=1)

In [14]:
oldcol = []
for i in df_cleaned.columns:
    oldcol.append(i)

In [16]:
columns_to_drop = ['S/N','is_non_empty', 'is_non_empty2','is_non_empty3',"Keywords_Present",'Keywords_Present2','Keywords_Present3'
                   , "Pt. Ref. No.","NAME","Invoice Period"]

df_cleaned = df_cleaned.drop(columns=columns_to_drop)

#lst = ['Year', 'Month', 'Day', 'CancerPatient', 'Age', 'Combined', 'Consultant', 'medicine2', 'Total Price', 'Hospitals', 'Patient', 'Gender','Qty','']
# Filter DataFrame to keep only specified columns
#df_cleaned = df_cleaned[lst]

In [17]:
df_cleaned['Combined'] = df_cleaned['Combined'].apply(normalise)

# Specify the path for the new Excel file
output_excel_path = 'Output/OCR_excel_LY.xlsx'

# Write the combined dataframe to a new Excel file
df_cleaned.to_excel(output_excel_path, index=False)

# Display a message indicating the success
print(f"Combined data has been written to {output_excel_path}")

Combined data has been written to Output/OCR_excel_LY.xlsx
